In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster import hierarchy as sch

In [2]:
data = preprocess_for_bow('data.csv', preprocessing=False)
bertopic=BERTopicWrapper(data['text'], data['ids'], custom_vectorizer=True)
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [3]:
linkage_function = lambda x: sch.linkage(x, 'average', optimal_ordering=True)
hierarchical_topics = bertopic.model.hierarchical_topics(data['text'], linkage_function=linkage_function)

100%|██████████| 675/675 [00:05<00:00, 129.26it/s]


In [4]:
bertopic.model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [19]:
indexes=[]
double=[]
for i in ["Virus Origin", "Virus contraction (who contracts the virus) and transmission (what transmits the virus)", 
                    "Incidence, Mortality and 'Casedemic'", "Prevention (Face Mask, Vitamin D, etc.)", 
                    "Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)", "Vaccines", "COVID-19 Denialism"]:
    print(i, bertopic.model.find_topics(i)[0], bertopic.model.find_topics("vaccines")[1])
    for j in bertopic.model.find_topics(i)[0]: 
        if j not in indexes:
            indexes.append(j)
        else:
            double.append(j)

Virus Origin [495, 87, 63, 135, 454] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]
Virus contraction (who contracts the virus) and transmission (what transmits the virus) [495, 454, 523, 593, 63] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]
Incidence, Mortality and 'Casedemic' [78, 519, 387, 107, 45] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]
Prevention (Face Mask, Vitamin D, etc.) [567, 2, 654, 277, 372] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) [51, 206, 259, 575, 433] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]
Vaccines [47, 36, 469, 576, 16] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]
COVID-19 Denialism [39, 74, 448, 128, 259] [0.7720082, 0.7150917, 0.6854383, 0.6851345, 0.6796175]


In [23]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in indexes]
len([data['text'][i] for i in idx_to_keep])

1444

In [21]:
double

[495, 454, 63, 259]

In [22]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [454]]
[data['text'][i] for i in idx_to_keep]

['Man spits in tea bottle and puts it back in supermarket to spread coronavirus',
 'The novel coronavirus can be spread through fruits and vegetables because it remains viable on them for 12 hours',
 'Tomato virus is affecting humans. More deadly than Coronavirus',
 'TikTok user Shadab Khan showing how to spread coronavirus',
 'A virus found affecting tomatoes in Maharashtra could be more dangerous than coronavirus and can kill humans.',
 'Eating infant flesh caused novel coronavirus outbreak',
 'Tomato virus is worse than coronavirus',
 'Man used injection filled with spit on fruits to spread coronavirus',
 'Coronavirus can spread through vegetables and fruits skin',
 'New mysterious virus attacking tomatoes is linked to the coronavirus, can be transmitted to humans upon consumption.',
 'Novel coronavirus transmitted through worms in food and drink?']

## virus origin

In [9]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [495, 87, 63, 135, 454]]
[data['text'][i] for i in idx_to_keep]

['In 2015 a ‘System and Method for Testing for COVID-19’ was patented by Richard Rothschild with a Dutch government organisation. Did you catch that? In 2015 – four years before the disease even existed – a testing method for COVID-19 was developed. (2B)',
 'Coronavirus is a FAMILY of viruses ranging from a cold to pneumonia. Covid-19 is specific. We were warned EARLY that a second wave of the flu would hit. It did. As usual. This is a gimmick being used to wage a war on the people. #QAnon #IwillNOTComply',
 'China made it. It\'s a biological agent to aid in population control. They have the vaccine for it as well and as soon as they have crippled enough of the world economy they will miraculously ""discover"" it and save the planet. #CoronavirusPandemic #CoronaVirusUpdate #coronavirus',
 'Interesting to find that this Pirbright institute seems to own the patent on some other nice supposed pandemics from the past. SO far PATENT for the CORONAVIRUS # 1010701B2 7720825B1 3172319B1THIS IS

## Virus contraction

In [10]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [495, 454, 523, 593, 63]]
[data['text'][i] for i in idx_to_keep]

['Coronavirus is a FAMILY of viruses ranging from a cold to pneumonia. Covid-19 is specific. We were warned EARLY that a second wave of the flu would hit. It did. As usual. This is a gimmick being used to wage a war on the people. #QAnon #IwillNOTComply',
 'I think there are many corona viruses. It looks like the ""tests"" suck. I\'ve studied history and have a SENSE of what an epidemic OR pandemic would look like - this is NONE of the former. You don\'t need vaccines for fake viruses. Finally: being over 80 is bad for your health.',
 'I wish people would stop calling it ""the Coronavirus"" there are many Coronaviruses such as the common cold. Call it what it is Covid19/sars-cov-2/Wuhan Bioweapon Disease: coronavirus disease2019 (COVID-19) Virus: severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)',
 "The purpose of inserting parts of HIV's genes into the #nCoV/#COVID19 bioweapon is so that it bonds more easily with receptor cells which is what makes this virus so infectious. 

## Incidence, Mortality and 'Casedemic'

In [11]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [78, 519, 387, 107, 45]]
[data['text'][i] for i in idx_to_keep]

['Do you support more deaths by Covid? Schools have opened only to close because of an increase of COVID! Just saw that more than 70 people now have Covid from that Sturgis bikers convention? What motivates your desire to see Americas die?',
 "Until Covid he had a roughly 60% chance to win. 2018 proved he had a major counter movement against him. 60% is a good chance. But then Covid hit and his chances fell to at best 40%. Still not bad but not odds I'd take in Vegas.",
 "Since the new World Order started the covid-19 global pandemic scare all natural deaths of the elderly the sick and The Dying was cancelled there hasn't been a single case of death by natural causes it's all been put down to covid-19 doctors in USA speaking out about now",
 'The #COVID_19italy has a mortality % of nearly 10%. That is 1 in 10 infections resulting in death. If the Chinese JapaneseTaiwanese succeed in tracking this back to Ft. Detrick bioweapons lab in US by sampling post mortem blood of false labeled de

## Prevention (Face Mask, Vitamin D, etc.)

In [12]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [567, 2, 654, 277, 372]]
[data['text'][i] for i in idx_to_keep]

['Fauci is just another Deep State operative. We’ve always known masks can’t filter virus particles larger than .3 microns. ( covid particles are .125 microns) but what they can do esp in uncontrolled environment & worn by untrained ppl is spread virus via fomite transmission',
 '....ELDERLY ... IT MAY PROTECT YOU SOME....OTHERWISE THE ONLY USE FOR THE MASK IS TO MIND CONTROL YOU WITH FEAR GENERATION SO THEY CAN STRIP YOU OF YOUR RIGHTS. COVID19 DID NOT AND IS NOT GOING TO KILL BY THE MASSES. THROW THE FACE MASK AWAY',
 'I haven’t seen any arrest for No face masks and no social distancing....so....this false flag has just broken the death grip you have on us law abiding citizens!!! Your comrades screwed you bad. 🤣 Riots have CURED YOUR COVID HOAX YOU MURDERER',
 'All on this thread.... sheeple so much for science about effectiveness of masks some studies say they work others show they’re useless even if everyone at the rally got Covid at .99% survival rate maybe 10 out of 1000 die mayb

## Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)

In [13]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [51, 206, 259, 575, 433]]
[data['text'][i] for i in idx_to_keep]

['Fauci needs to answer why he withheld his brief describing Hydroxychloroquine cured covid. This man is a fraud. Bright fights against it too! Why do they want more people to die? Big Pharma money? Deep state instructions? De-population?',
 'I would fire him now! 2013 he advocated for hydrochloroquine but now stands eerily silent! He has not advised POTUS properly in any part of this pandemic! He is too busy pandering to the deep state to care about American lives lost and American livelihoods destroyed!',
 "Hydroxychloroquine is not an antiviral it is an antimalarial drug Science isn't 'silly or ideological' it is based on hypothesis testing - the hypothesis that HQ was an effective treatment for #Covid19 was tested and it was found to have no effect As I say sorry to disappoint",
 "Trump is still trying to sell hydroxychloroquine to the sheeple. Just told White House reporters that he's been taking it for a week and a half despite testing negative for COVID-19... One has to wonder i

## Vaccines

In [14]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [47, 36, 469, 576, 16]]
[data['text'][i] for i in idx_to_keep]

['Oh I beg to differ. The genetic code of the mrna in this vaccine is different from the viral DNA encapsulated within our own DNA which involved the implantation of embryos. This will have zero effect on that gene and this is known because women who have had covid have had no',
 'I gave out a little #RedPill while waiting in a q today. Guy saying all will be better with covid once we get vaccine. I gave him a little list of what was in them and said I wont be having 1. He said ""I didnt know that. Thank u very much"" #WWG1WGAWORLDWIDE',
 'that could render people barren/infertile or cause birth defects or create an even worse Covid. The reason they are having such a hard time now with a vaccine is because it mutates at an exponential rate. I have no problem with a vaccine IF it has gone through the proper vetting',
 'As a medical doctor of course I support vaccinations. I have a problem with the FDA being controlled by drug companies. — Dr. Jill Stein (@DrJillStein) July 29, 2016',
 '

## COVID 19 denialism

In [15]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in [39, 74, 448, 128, 259]]
[data['text'][i] for i in idx_to_keep]

['CDC REFUSING ANY TESTING AT HOSPITALS EVEN WITH SYMPTOMS OF THE COVID19 UNLESS YOU JUST RETURNED FROM OVERSEAS. THIS IS A SERIOUS SITUATION NO ONE GETTING TESTED ISNT THIS OBVIOUS THEIR PLAN PANDEMIC FOR DEPOPULATION EXISTS HAPPENING NOW!',
 'Americans deserve to know who was involved in unleashing the man-made COVID-19 virus on us (& around the world.) We need to know. The Democrats the propagandist fear-mongering media & their deep state puppet masters cannot overcome good with their evil.',
 "It's not that Covid isn't real it's that Covid is't something to destroy our lives and change the every fabric our lifestyles. This is a draconian power grab from money hungry individuals compromised by communist china to establish the Great Reset for a new world order..",
 "Lol it makes sense. Just because you can't comprehend it doesn't mean its senseless. What doesn't make sense is that i made zero comment about it being ok to get covid and somehow you managed to get that out of what I sai